Sprint 4: Analyzing Car Advertising Dataset

Introduction: This project is to analyze a dataset of cars that is collected from 1970s-2010s. I will analyze the most common types of vehicles, how much revenue each year made, and the correlation between the numbers of cyclinder and cost.

In [1]:
import pandas as pd
import numpy
import streamlit as st
import plotly_express as px

In [2]:
import plotly.io as pio
pio.templates.default = 'plotly'

In [3]:
st.header('Sprint 4 Project: Analyzing Car Advertisment Dataset')

2023-03-08 01:49:20.628 
  command:

    streamlit run /Users/quynhnguyen/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [4]:
df =pd.read_csv('vehicles_us.csv')

In [5]:
df.head()


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [7]:
df.isnull().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

In [8]:
model_year_median = df['model_year'].median()
cylinder_median = df['cylinders'].median()
odometer_median = df['odometer'].median()
print('model year:', model_year_median)
print('cylinders:' ,cylinder_median)
print('odometer:', odometer_median)

model year: 2011.0
cylinders: 6.0
odometer: 113000.0


In [9]:
# replacing missing values with 0 
df['model_year']=df['model_year'].fillna(df.groupby('model')['model_year'].transform('median'))
df['condition']=df['condition'].fillna('unkown')
df['cylinders']=df['cylinders'].fillna(df.groupby('condition')['cylinders'].transform('median'))
df['odometer']=df['odometer'].fillna(df.groupby('fuel')['odometer'].transform('median'))
df['paint_color']=df['paint_color'].fillna('unknown')
df['is_4wd']=df['is_4wd'].fillna(0)
df.isnull().sum()

price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64

In [10]:
#changing appropriate columns into datetime 
df['date_posted']=pd.to_datetime(df['date_posted'], format='%Y-%m-%d')


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    51525 non-null  float64       
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     51525 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      51525 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   51525 non-null  object        
 10  is_4wd        51525 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(6)
memory usage: 5.1+ MB


In [12]:
df['type'].value_counts()

SUV            12405
truck          12353
sedan          12154
pickup          6988
coupe           2303
wagon           1541
mini-van        1161
hatchback       1047
van              633
convertible      446
other            256
offroad          214
bus               24
Name: type, dtype: int64

In [13]:
car_price = df.groupby('type').agg({'price': 'mean'}).sort_values(by= ['price'], ascending= False).reset_index()
px.histogram(car_price, y='price', x='type', color='type')


Bus, truck and pickup cost the most on average. Mini-van, sedan, and hatchback cost less on average. This can be explained by their small size. 

In [14]:
year = df.groupby('model_year').agg({'price': 'sum'}).reset_index()
year1 = year.drop(index = 0) #removing car that doesnt have model_year
year1


,model_year,price
1,1929.0,18000
2,1936.0,5000
3,1948.0,21000
4,1949.0,44900
5,1954.0,15000
...,...,...
65,2015.0,62251781
66,2016.0,58870298
67,2017.0,50571634
68,2018.0,54378759


In [15]:
car_sales_2000s = year1[year1['model_year'] >= 2000.0]
car_sales_1900s = year1[year1['model_year'] <= 2000.0]

In [16]:
scatter_2000s = px.scatter(car_sales_2000s, y='price', x='model_year', title='Cars Revenue in the 2000s', color = "model_year")
scatter_1900s = px.scatter(car_sales_1900s, y='price', x='model_year', title='Cars Revenue in the 1900s', color='model_year')
scatter_2000s.show()
scatter_1900s.show()

In [17]:
px.scatter(year1, y='price', x='model_year', title= 'Car sales per Year', color='model_year')

from the scatterplot, most car sales and productions were made around 2018s. Before 2000, it looks like people were using other modes of transporation. 

In [18]:
condition = df.groupby('condition').agg({'days_listed': 'mean'}).reset_index()
condition

,condition,days_listed
0,excellent,39.611714
1,fair,39.118233
2,good,39.631323
3,like new,39.166807
4,new,37.111888
5,salvage,39.008696


In [19]:
px.histogram(condition, x='condition', y='days_listed', color='condition')

The average number of days to sell cars, for all condition, is 39 days. 

In [20]:
cylinder = df.groupby('cylinders').agg({'price':'mean'}).reset_index()
cylinder1 = cylinder.drop(index=1)
cylinder1

,cylinders,price
0,3.0,8571.911765
2,5.0,6602.915441
3,6.0,12310.264933
4,8.0,15881.309455
5,10.0,7237.244080
6,12.0,19000.000000


In [21]:
px.scatter(cylinder1, x='cylinders', y='price', color='cylinders')

There is an increased in price as the number of cyclinder increases. Except for 0 and 10 cyclinders, these can be considered as outliers. 

Conclusion: Analysis of car prices, car sales, how long it took to sell a car, and cyclinders were done. From the first graph, we see that bus, pickup, and truck cost the most, on average. Whereas smaller cars cost less. From graph 2, we see that car sale produce thousands of revenues in 1900s to billions in 2000s. However, there was a drop in car sale in 2019. Another analysis that was made was to see the average days it took to sell cars with different conditions. It takes 39 days, on average, in every condition. Cyclinder cost increases as the number of cyclinder increases, with outlier for 0 and 10 cyclinders. 